<a href="https://colab.research.google.com/github/nabeelsana/Nabeel_NLP_Projects/blob/main/Voice_to_text_using_Tensorflow_Using_MFCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio

In [ ]:
import numpy as np
import sklearn
import csv
import os
import pandas as pd

In [ ]:
import tensorflow as tf
tf.__version__

'2.11.0'

In [ ]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.4 MB/s eta 0:00:00


In [ ]:
from jiwer import wer

# DATASET LINK

https://keithito.com/LJ-Speech-Dataset/

In [ ]:
#Mounting Google drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# STEP 1: Listing all audio clips and verifying their presence, then extract file names without extension

In [ ]:
#SKIP
#files = os.listdir('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/wavs')

print(type(files))
print(files[:10])


KeyboardInterrupt: ignored

In [ ]:
#SKIP
#storing file names in drive as a csv file

with open('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/LJclipnames.csv', "w", newline = '') as file:
  writer = csv.writer(file)
  for name in files:
    writer.writerow([name])         #[name] will print each name as a new row

In [ ]:
#Importing file as a dataframe

clips_df = pd.read_csv('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/LJclipnames.csv')

In [ ]:
clips_df.columns = (['Clips_name'])
print(len(clips_df))
print(clips_df.head())

13099
       Clips_name
0  LJ007-0088.wav
1  LJ002-0275.wav
2  LJ016-0071.wav
3  LJ015-0012.wav
4  LJ018-0279.wav


In [ ]:
clips_df['file_name']= clips_df['Clips_name']

In [ ]:
clips_df.head()

,Clips_name,file_name
0,LJ007-0088.wav,LJ007-0088.wav
1,LJ002-0275.wav,LJ002-0275.wav
2,LJ016-0071.wav,LJ016-0071.wav
3,LJ015-0012.wav,LJ015-0012.wav
4,LJ018-0279.wav,LJ018-0279.wav


In [ ]:
clips_df[['file_name', 'Extension']] = clips_df['file_name'].str.split('.',1, expand = True)

In [ ]:
print(clips_df.head())
print(len(clips_df))

       Clips_name   file_name Extension
0  LJ007-0088.wav  LJ007-0088       wav
1  LJ002-0275.wav  LJ002-0275       wav
2  LJ016-0071.wav  LJ016-0071       wav
3  LJ015-0012.wav  LJ015-0012       wav
4  LJ018-0279.wav  LJ018-0279       wav
13099


In [ ]:
#STEP 2: Read Meta data file contains clips names and sentences

In [ ]:
meta_df = pd.read_csv('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/metadata.csv',sep = '|', header = None, )  #make sure file is UTF8 encoded



In [ ]:
meta_df.columns = (['file_name', 'raw_transcription', 'transcription'])

In [ ]:
meta_df.head(5)

,file_name,raw_transcription,transcription
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...


In [ ]:
meta_df[meta_df['transcription'].isnull()]   # To check if any column in Trabscription is Null , will drop these lines

,file_name,raw_transcription,transcription
945,LJ004-0076,"Disease, cold, famine, nakedness, and contagio...",NaN
948,LJ004-0079,All measures and practices in prison which may...,NaN
1119,LJ004-0250,It is obvious that reformation must be materia...,NaN
1147,LJ005-0028,"There should be no tea and sugar, no assemblag...",NaN
1149,LJ005-0030,"In prisons, which are really meant to keep the...",NaN
1196,LJ005-0077,expedient to introduce such measures and arran...,NaN
1371,LJ005-0253,to inquire into and report upon the several ja...,NaN
1400,LJ005-0282,"the comparatively innocent are seduced, the un...",NaN
1500,LJ006-0083,"So great is the authority exercised by him,|So...",NaN
1655,LJ006-0238,There was a disturbance in the transport yard ...,NaN


In [ ]:
meta_df.dropna(inplace= True)  # drop those lines where transcription column is Null

In [ ]:
len(meta_df)

13084

In [ ]:
#STEP 3: Compare clips df with Meta df and drop lines in meta file that are not present as clip

In [ ]:
metadata_df = pd.merge(clips_df, meta_df, on = 'file_name')

In [ ]:
print(len(metadata_df))
print(metadata_df.head(1))
print(metadata_df.columns)

13083
       Clips_name   file_name Extension  \
0  LJ007-0088.wav  LJ007-0088       wav   

                               raw_transcription  \
0  for not less a period than twenty-four years.   

                                   transcription  
0  for not less a period than twenty-four years.  
Index(['Clips_name', 'file_name', 'Extension', 'raw_transcription',
       'transcription'],
      dtype='object')


In [ ]:
#Just for comfort write file as xls

In [ ]:
#metadata_df.to_excel('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/df.xlsx')


In [ ]:
#Step 4: Get metadata working file

In [ ]:
meta_df = metadata_df[['Clips_name', 'file_name', 'transcription']]

In [ ]:
meta_df = meta_df.sample(frac= 1).reset_index(drop= True)

In [ ]:
meta_df.head(5)

,Clips_name,file_name,transcription
0,LJ032-0166.wav,LJ032-0166,Judgments as to probability will depend on the...
1,LJ045-0158.wav,LJ045-0158,He spent quite a bit of time putting away diap...
2,LJ009-0303.wav,LJ009-0303,"Then came the cost of the gallows,"
3,LJ004-0214.wav,LJ004-0214,"Its evils were inherent and irremediable, but ..."
4,LJ012-0076.wav,LJ012-0076,There was an increase rather than an abatement...


In [ ]:
#meta_df = meta_df[:100]   # initial testing of model with 100 examples

In [ ]:
len(meta_df)

13083

In [ ]:
#Step 5: Split into Training Validation data set

In [ ]:
split =int(len(meta_df)*0.9)
print(split)

11774


In [ ]:
df_train = meta_df[:split]
df_val = meta_df[split:]
print(len(df_train))
print(len(df_val))

11774
1309


In [ ]:
#MACHINE LEARNING PROCESS

In [ ]:
path = '/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/wavs/'

In [ ]:
#Functions to process input sentence

In [ ]:
characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]

In [ ]:
print(characters)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', ' ']


In [ ]:
char_to_num = tf.keras.layers.StringLookup(vocabulary = characters, oov_token= "")

In [ ]:
print(char_to_num.get_vocabulary())

['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', ' ']


In [ ]:
print(char_to_num.vocabulary_size())

31


In [ ]:
output_dim = char_to_num.vocabulary_size()+1

In [ ]:
print(output_dim)

32


In [ ]:
num_to_char = tf.keras.layers.StringLookup(vocabulary = char_to_num.get_vocabulary(), oov_token= "", invert = True)

In [ ]:
#Function to fetch audio files, create spectrogram and process corresponding sentences

In [ ]:
frame_length = 256
frame_step = 160
fft_length = 384

# frame length   = no of data points in each sliding window here 256 points will be covered in each frame out of 212893 input data point
# frame steps/hop size = no of data points to skip when sliding window so first 256 data points then skip 160 then start from data point 416 to 672, then skip another 160 and so on
#212893/frame steps = no of row  here 212893/160 =1329
#the size of the FFT (here 384) determines the number of frequency bins used to represent the spectrum of each frame of the signal.
#The number of frequency bins in the resulting spectrum is equal to half the FFT size plus one. so 384/2+1 = 193
#This means each row with contain  193 data points i.e columns

In [ ]:
def encode_single_sample(audio_file, label):
  # 1) process audio generating spectrogram
  #A reading and decoding file and change datatype to float

  file = tf.io.read_file(path+audio_file)
  audio,samplerate = tf.audio.decode_wav(file)
  audio = tf.squeeze(audio, axis = -1)
  audio = tf.cast(audio, tf.float32)

  #B Getting Spectrograph
  stfts = tf.signal.stft(audio, frame_length = frame_length, frame_step = frame_step, fft_length= fft_length)
  spectrograph = tf.abs(stfts)

  #C Warp the linear scale spectrograms into the mel-scale.

  num_spectrogram_bins = stfts.shape[-1]

  lower_freq = 80.0
  upper_freq = 7600.0
  num_mel_bins = 80

  linear_to_mel_weight_matrix= tf.signal.linear_to_mel_weight_matrix(num_mel_bins=num_mel_bins,
                                                                   num_spectrogram_bins = num_spectrogram_bins,
                                                                   sample_rate = samplerate,
                                                                   lower_edge_hertz = lower_freq,
                                                                   upper_edge_hertz= upper_freq)

  mel_spectrogram = tf.tensordot(spectrograph, linear_to_mel_weight_matrix, 1 )

  mel_spectrogram.set_shape(spectrograph.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:]))

  #D Compute a stabilized log to get log-magnitude mel-scale spectrograms.

  log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)

  #Compute MFCC

  mfcc = tf.signal.mfccs_from_log_mel_spectrograms (log_mel_spectrogram)[..., :13] # take first freq 13 bins   #https://www.tensorflow.org/api_docs/python/tf/signal/mfccs_from_log_mel_spectrograms

  #mfcc = tf.signal.mfccs_from_log_mel_spectrograms (log_mel_spectrogram)


  # Normalization

  means = tf.math.reduce_mean(mfcc, 1, keepdims = True)
  stddev = tf.math.reduce_std(mfcc, 1, keepdims = True)
  mfccs = (mfcc - means)/(stddev+ 1e-10)

  #2) process sentence

  label = tf.strings.lower(label)
  label = tf.strings.unicode_split(label,input_encoding = "UTF-8")
  label = char_to_num(label)

  return mfccs, label




In [ ]:
#Creating Dataset Object   # If any of column has null value , will give error.


In [ ]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((list(df_train['Clips_name']), list(df_train['transcription'])))

train_dataset = train_dataset.map(encode_single_sample, num_parallel_calls = tf.data.AUTOTUNE).padded_batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((list(df_val['Clips_name']), list(df_val['transcription'])))
val_dataset = val_dataset.map(encode_single_sample, num_parallel_calls = tf.data.AUTOTUNE).padded_batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
for batch in train_dataset:
  a, b = batch
  print(a.shape, b.shape)   # 32 clips, 1371 rows/timesteps, 80 freq each
  break

(32, 1261, 13) (32, 139)


In [ ]:
a[2].shape

TensorShape([1390, 80])

In [ ]:
for batch in val_dataset:
  x, y = batch
  print(x.shape, y.shape)
  break

(32, 1337, 80) (32, 156)


In [ ]:
#Machine Learning Model

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, ReLU, Reshape, Dropout, GRU, Reshape, Bidirectional

In [ ]:
model = Sequential([
    Reshape((-1, 13, 1), name="expand_dim", input_shape=(None, 13)),     # see explanation below                    (batch, rows, columns/data point in each row,      channels)
    #                                                                                                                   (-1, -1,         80,                            1) -1 means variable number
    Conv2D(filters = 32, kernel_size = [11,41], strides = 2, padding = 'same', use_bias = False,  name = 'conv_1'), # (-1,    -1,     80/2 = 40,                        32)
    BatchNormalization(name = 'conv_1_bn'),
    ReLU(name= 'conv_1_relu'),
    Conv2D(filters = 32, kernel_size = [11,41], strides = 2, padding = 'same', use_bias = False,  name = 'conv_2'),  # (-1,     -1,     40/2 = 20,                        32)
    BatchNormalization(name = 'conv_2_bn'),
    ReLU(name= 'conv_2_relu'),
    Reshape((-1,4*32)), #   (-1,-1,  columns*filters )
    #The RNN expects that the input is going to be sequential. Therefore, the input has the shape  [batch_size, time, feature_size]
    #reshape as [b_size, num_frames, am_width * am_height * num_filters]
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_1')), name = 'bidir_1' ),
    Dropout(rate = 0.5, name = 'drop_out_1'),
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_2')),  name = 'bidir_2' ),
    Dropout(rate = 0.5, name = 'drop_out_2'),
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_3')),  name = 'bidir_3' ),
    Dropout(rate = 0.5, name = 'drop_out_3'),
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_4')),  name = 'bidir_4' ),
    Dropout(rate = 0.5, name = 'drop_out_4'),
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_5')),  name = 'bidir_5' ),
    Dense(units= 512*2, name = 'dense1'),
    ReLU(name= 'Dense_1_relu'),
    Dropout(rate = 0.5, name = 'drop_out_5'),
    Dense(units = output_dim, activation = 'softmax')



])

In [ ]:
"""
input_shape=(None, 193) is a parameter used in defining the shape of the input data for a machine learning model.

In this case, None indicates that the input can have a variable number of rows (the first dimension), and 193 indicates that the input data has 193 columns (the second dimension).
The first dimension is often left as None to allow for variable batch sizes during training or inference.

For example, if we have a dataset of audio recordings, the first dimension of the input might correspond to the number of time steps in the recording,
while the second dimension might correspond to the number of frequency bins in a spectrogram representation of the audio signal.
In this case, input_shape=(None, 193) would be appropriate, as the length of the audio recordings can vary, but the number of frequency bins is fixed at 193.
If the input shape is (None, 193), applying the Reshape layer with Reshape((-1, 193, 1)) will transform the input into a new shape of (None,None, 193, 1).

Here, -1 is a special value that indicates that the size of the first dimension of the input should be inferred from the remaining dimensions and the batch size.
In other words, the number of samples in the batch can vary, but the size of the other dimensions remains constant.

The resulting shape (None,None 193, 1) represents a 4D tensor, where the first dimension is the batch size, the second dimension is the number of time steps (rows), and the third dimension is freq bins and 4th dim is
 the number of channels
(which is 1 in this case). This 4D tensor is commonly used as input for 2D convolutional neural networks (CNNs) that are designed to operate on sequences of data,
 such as time series or audio signals.

"""

'\ninput_shape=(None, 193) is a parameter used in defining the shape of the input data for a machine learning model.\n\nIn this case, None indicates that the input can have a variable number of rows (the first dimension), and 193 indicates that the input data has 193 columns (the second dimension). The first dimension is often left as None to allow for variable batch sizes during training or inference.\n\nFor example, if we have a dataset of audio recordings, the first dimension of the input might correspond to the number of time steps in the recording, while the second dimension might correspond to the number of frequency bins in a spectrogram representation of the audio signal. In this case, input_shape=(None, 193) would be appropriate, as the length of the audio recordings can vary, but the number of frequency bins is fixed at 193.\nIf the input shape is (None, 193), applying the Reshape layer with Reshape((-1, 193, 1)) will transform the input into a new shape of (None, 193, 1).\n\

In [ ]:
 #Convolution output:
    #The number of filters (32) corresponds to the number of output channels.
    #The kernel size of [11, 41] indicates that the convolution will be done over an 11x41 window.
    #The stride of 2 indicates that the filter will move 2 pixels at a time.
    #The padding of 'same' means that the input will be padded with zeros to ensure that the output has the same spatial dimensions as the input.
    #With these parameters, the convolution operation will reduce the spatial dimensions of the input by a factor of 2 in both the vertical and horizontal directions
    #(since the stride is 2), while increasing the number of channels to 32.
    #output will be of shape (-1,-1,  columns/2 rounded to nearest whole,filters )

In [ ]:
"""
explanation for first reshape
The Reshape layer is used to reshape the input tensor into a new shape. In this case, the layer will reshape the input tensor to have the following dimensions:

The first dimension will be set to -1, which means that it will be dynamically computed based on the size of the input tensor and the other dimensions specified in the reshape.
The second dimension will be set to 1, indicating that each element in the input tensor will be reshaped into a 1-dimensional vector.
The third dimension will be set to input_dim, which is the size of the input data dimension.
The fourth dimension will be set to 1, indicating that each element in the reshaped tensor will have a single channel.
Overall, this layer will transform the input tensor into a 4-dimensional tensor with a variable number of rows, a single column, input_dim channels, and a single batch dimension. This can be useful, for example, when preparing image data for input to a convolutional neural network.*/
"""


'\nexplanation for reshape\nThe Reshape layer is used to reshape the input tensor into a new shape. In this case, the layer will reshape the input tensor to have the following dimensions:\n\nThe first dimension will be set to -1, which means that it will be dynamically computed based on the size of the input tensor and the other dimensions specified in the reshape.\nThe second dimension will be set to 1, indicating that each element in the input tensor will be reshaped into a 1-dimensional vector.\nThe third dimension will be set to input_dim, which is the size of the input data dimension.\nThe fourth dimension will be set to 1, indicating that each element in the reshaped tensor will have a single channel.\nOverall, this layer will transform the input tensor into a 4-dimensional tensor with a variable number of rows, a single column, input_dim channels, and a single batch dimension. This can be useful, for example, when preparing image data for input to a convolutional neural network.

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)

In [ ]:
#Defining CTC Loss Function
def CTCLoss(y_true, y_pred):
  batch_length = tf.cast(tf.shape(y_true)[0], dtype= 'int64')
  input_length = tf.cast(tf.shape(y_pred)[1], dtype= 'int64' )
  input_length = input_length*tf.ones(shape = (batch_length,1),dtype= 'int64' )

  label_length = tf.cast(tf.shape(y_true)[1], dtype= 'int64' )
  label_length = label_length*tf.ones(shape = (batch_length,1),dtype= 'int64' )




  loss = tf.keras.backend.ctc_batch_cost(y_true,y_pred, input_length, label_length)  #https://keras.io/ko/backend/  for ctc_batch_cost
  return loss


In [ ]:
model.compile(optimizer = opt, loss = CTCLoss)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 expand_dim (Reshape)        (None, None, 13, 1)       0         
                                                                 
 conv_1 (Conv2D)             (None, None, 7, 32)       14432     
                                                                 
 conv_1_bn (BatchNormalizati  (None, None, 7, 32)      128       
 on)                                                             
                                                                 
 conv_1_relu (ReLU)          (None, None, 7, 32)       0         
                                                                 
 conv_2 (Conv2D)             (None, None, 4, 32)       461824    
                                                                 
 conv_2_bn (BatchNormalizati  (None, None, 4, 32)      128       
 on)                                                  

In [ ]:
hitory = model.fit(train_dataset, validation_data = val_dataset ,epochs = 10)

Epoch 1/10
368/368 [==============================] - 2266s 6s/step - loss: inf - val_loss: 338.2608
Epoch 2/10
368/368 [==============================] - 1357s 4s/step - loss: inf - val_loss: 339.0825
Epoch 3/10
368/368 [==============================] - 1340s 4s/step - loss: inf - val_loss: 318.0742
Epoch 4/10
368/368 [==============================] - 1337s 4s/step - loss: inf - val_loss: 333.8138
Epoch 5/10
368/368 [==============================] - 1328s 4s/step - loss: inf - val_loss: 266.5611
Epoch 6/10
368/368 [==============================] - 1429s 4s/step - loss: inf - val_loss: 228.6001
Epoch 7/10
368/368 [==============================] - 1406s 4s/step - loss: inf - val_loss: 216.0003
Epoch 8/10
368/368 [==============================] - 1375s 4s/step - loss: inf - val_loss: 195.9469
Epoch 9/10
368/368 [==============================] - 1376s 4s/step - loss: inf - val_loss: 185.1549
Epoch 10/10
368/368 [==============================] - 1362s 4s/step - loss: inf - val_loss

In [ ]:
#Defining Prediction decoding function

def decode_batch_prediction(pred):
  input_length = np.ones(pred.shape[0])*pred.shape[1]
  results = tf.keras.backend.ctc_decode(pred, input_length = input_length, greedy = True)[0][0]                     # https://keras.io/ko/backend/ (for CTC_decode)
  output_text = []
  for result in results:
    result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
    output_text.append(result)
  return output_text



In [ ]:
#Inferencing

predictions = []
targets = []

for batch in val_dataset:
  x,y = batch
  batch_predictions = model.predict(x)
  batch_predictions = decode_batch_prediction(batch_predictions)
  predictions.extend(batch_predictions)

  for label in y:
    label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
    targets.append(label)

wer_score = wer(targets, predictions)
print(f"WER: {wer_score:.4f}")

for i in np.random.randint(0,len(predictions),5):
  print(f"Target :{targets[i]}")
  print(f"Predictions :{predictions[i]}")






1/1 [==============================] - 4s 4s/step
WER: 0.8006
Target :would be astonished to observe the peculiar tenderness i was going to add respect
Predictions :whl he istonie to oserf potoer tenders w  a e
Target :some of the officers saw oswald strike mcdonald with his fist most of them heard a click which they assumed to be a click of the hammer of the revolver
Predictions :some of the officers sa oswald stritmi dold whith hi sfeist most ovem per apleck was n t te     
Target :this committee as well as the one preceding it also reported in terms of strong reprobation on the small prisons and jails
Predictions :this commat as walas the on precenet aso reportedginturs of strom reprpaion o os as s s
Target :and if we determined that the course of the investigation required us to clarify or face him down with this information we would do it at the appropriate time
Predictions :and ifwe atermened that the corse of the investigation wreqarred ustclarfi r faion down whith his imfermati

In [ ]:
predictions

['i segetof maantud an importens sofficion to exploe other jails e en n n r re e ',
 'he ad not eanpo strier e cas of consr o ren en n ',
 'in the mal abot chawa be wen buingon in sn n s s',
 'ie vestigation determent that this potagraf was tak an apromently seven tentsof he moo wr r',
 'he was takin to a osl in mos ower ie was captinto oto rtwenty i en n',
 'the bol of the prisonrs wer stil ltn ns',
 'o don a triet o perwatems ceny to lethe aria bat sh re fs e e e e e  ',
 'i masofting non ha opaticunally his',
 'foler men wil lain newgate no harded with other onnen r s ',
 'mers my paid n oe oe r',
 'at the firstnterigation oswald clamed that his only crime was harin nm mn n ',
 'he remo at lease som of the piturs whichadbeen pastedinit and same dem a mon his affas w rer e    ',
 'aqor assas execution in ate fre s s ',
 'any presidence have ban under standibly impation whith the securty procationsw he e e e e e e',
 'pimes tobeu micals offics managur of o tred rosin compnincorpry in 